In [ ]:
# API WoS v.1 (unofficial https://github.com/enricobacis/wos/tree/master)

from wos import WosClient
import wos.utils

with WosClient(SID='f312e2fe4d304f467ec3bd7d2af46ea75c5817d7') as client:
    print(wos.utils.query(client, 'AU=Knuth Donald'))

In [ ]:
# API WoS v.2 (official https://github.com/clarivate/wosstarter_python_client)
# Package outdated (setup.py)

import time
import clarivate.wos_starter.client
from pprint import pprint
from clarivate.wos_starter.client.apis.tags import documents_api
from clarivate.wos_starter.client.model.document import Document
from clarivate.wos_starter.client.model.documents_list import DocumentsList

In [ ]:
# Homemade WoS API (scratch scrapping)


# If the request is blocked by Clavirate,
# this code might be useful to change the user agent 

# from fake_useragent import UserAgent
# ua = UserAgent()
# headers = {'User-Agent': ua.random}
# response = requests.get(url, headers=headers)

import os
import shutil
import time
import pytz
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By


def get_papers_metadata(plateform: int, query: str):
    
    """Get metadata (title, authors, DOI, etc.) of papers coming from scientific literature platforms

    This function provides metadata of scientific articles coming from the
    results of a searching query on scientific literature platform. The
    metadata collected for each article are:
    * DOI
    * Title
    * Authors
    * Publication year
    * Abstract
    * Keywords
    * Language
    * Affiliation
    * Journal
    * Publisher
    * Volume
    * Issue
    The platforms supported are:
    * Web of Science (https://www.webofscience.com/wos/woscc/advanced-search)

    Parameters:
    plateform (int): platform identification code
                     0: Web of Science
    query (str): searching query (must respect the sythax of the platform used)

    Returns:
    xxx

    """

    utc_current_time = datetime.datetime.now(tz=pytz.timezone('UTC'))
    utc_current_time_str = str(utc_current_time.year) + "-" + str(utc_current_time.month) + "-" + str(utc_current_time.day) + "_" + str(utc_current_time.hour) + "-" + str(utc_current_time.minute) + "-" + str(utc_current_time.second)
    
    match plateform:

        case 0:
            
            url = "https://www.webofscience.com/wos/woscc/advanced-search"
            driver = webdriver.Firefox() # manage various web browser
            driver.get(url)

            time.sleep(3)
            driver.find_element(By.ID, "onetrust-reject-all-handler").click()
            
            time.sleep(2)
            search_input = driver.find_element(By.ID, "advancedSearchInputArea")
            search_input.send_keys(query)
            search_input.submit()

            time.sleep(2)
            nb_results = driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/div[1]/h1/span").text
            nb_results = int(nb_results.replace(',', ''))

            for batch_nb in range(int(int(nb_results)/1000)):
                
                time.sleep(2)
                driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[1]/div/app-export-option/div/app-export-menu/div/button").click()
                
                time.sleep(1)
                driver.find_element(By.ID, "exportToExcelButton").click()

                time.sleep(1)
                records_from_button = driver.find_element(By.ID, "radio3-input")
                driver.execute_script("arguments[0].click();", records_from_button)

                low_bound = driver.find_element(By.ID, "mat-input-"+str(2*batch_nb+1))
                low_bound.clear()
                low_bound.send_keys(str(int(batch_nb*1000) + 1))
                high_bound = driver.find_element(By.ID, "mat-input-"+str(2*batch_nb+2))
                high_bound.clear()
                high_bound.send_keys(str(int(batch_nb*1000) + 1000))

                driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button").click()
                driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/div/div/div/div[3]").click()
                driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]").click()
            
                time.sleep(10)
                downloaded_files = os.listdir("/home/er/Downloads")
                full_path_downloaded_files = []
                for i in downloaded_files:
                    full_path_downloaded_files.append("/home/er/Downloads/"+i)
                latest_file = max(full_path_downloaded_files, key=os.path.getctime)
                output_path = "./data/papers_metadata/" + utc_current_time_str
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                shutil.move(latest_file, os.path.join(output_path, str(batch_nb + 1) + ".xls"))

            time.sleep(2)
            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[1]/div/app-export-option/div/app-export-menu/div/button").click()
            
            time.sleep(1)
            driver.find_element(By.ID, "exportToExcelButton").click()

            time.sleep(1)
            records_from_button = driver.find_element(By.ID, "radio3-input")
            driver.execute_script("arguments[0].click();", records_from_button)

            low_bound = driver.find_element(By.ID, "mat-input-"+str(2*int(int(nb_results)/1000)+1))
            low_bound.clear()
            low_bound.send_keys(str(int(int(nb_results)/1000)*1000))
            high_bound = driver.find_element(By.ID, "mat-input-"+str(2*int(int(nb_results)/1000)+2))
            high_bound.clear()
            high_bound.send_keys(str(nb_results))

            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button").click()
            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/div/div/div/div[3]").click()
            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]").click()
        
            time.sleep(10)
            downloaded_files = os.listdir("/home/er/Downloads")
            full_path_downloaded_files = []
            for i in downloaded_files:
                full_path_downloaded_files.append("/home/er/Downloads/"+i)
            latest_file = max(full_path_downloaded_files, key=os.path.getctime)
            output_path = "./data/papers_metadata/" + utc_current_time_str
            if not os.path.exists(output_path):
                os.makedirs(output_path)
            shutil.move(latest_file, os.path.join(output_path, str(int(int(nb_results)/1000)+1) + ".xls"))


        # case 1:
        #     xxx

        # case _:
        #     # manage error
    
get_papers_metadata(0, "ts = 'soil organic carbon'")

# "TS = ('meta*analysis' (OR) 'systematic review') (AND) ('soil organic carbon' (OR) 'SOC' (OR) 'soil organic matter' (OR) 'SOM' (OR) 'soil carbon')"


In [ ]:
import os
import pandas as pd

xls_files = '/home/er/Documents/Cirad/SOCSciCompiler/data/papers_metadata/2023-7-3_11-24-45'

data = []

for filename in os.listdir(xls_files):
    if filename.endswith('.xls'):
        file_path = os.path.join(xls_files, filename)
        df = pd.read_excel(file_path)
        data.append(df)

merged_df = pd.concat(data)

pickle_file = '2023-7-3_11-24-45_metadata.pkl'
merged_df.to_pickle(pickle_file)

print(f"Merged data saved to {pickle_file}")


In [ ]:
pickle_file = '2023-7-3_11-24-45_metadata.pkl'
df = pd.read_pickle(pickle_file)

attributes_to_keep = ['DOI'
                     ,'Article Title'
                     , 'Authors'
                     , 'Publication Year'
                     , 'Abstract'
                     , 'Author Keywords'
                     , 'Language'
                     , 'Affiliations'
                     , 'Source Title'
                     , 'Publisher'
                     , 'Volume'
                     , 'Issue']

df = df[attributes_to_keep]
print("\nSummary Information:")
print(df.info())

df = df.drop_duplicates()
print("\nSummary Information:")
print(df.info())

df = df[df['DOI'].duplicated(keep=False) == False]
print("\nSummary Information:")
print(df.info())

df.to_pickle('2023-7-3_11-24-45_metadata.pkl')

In [ ]:
(86993 - 81702)*100/86993

In [ ]:
batch_nb_values = list(range(87))
tmp_values = list(range(1, 88, 2))
button_id_values = []
for i in tmp_values:
    button_id_values.append([tmp_values[i], tmp_values(i+1)])
for i in batch_nb_values:
    print(batch_nb_values[i], button_id_values[i])

In [ ]:

latest_file = max(full_path_downloaded_files, key=os.path.getctime)
shutil.move(latest_file, "./data/papers_metadata/" + utc_current_time_str + "/" + str(batch_nb) + ".xls")

output_path = "./data/papers_metadata/" + utc_current_time_str
if not os.path.exists(output_path):
    os.makedirs(output_path)
shutil.move(latest_file, os.path.join(output_path, str(batch_nb) + ".xls"))

In [2]:
# API Sci-Hub (https://github.com/zaytoun/scihub.py/tree/master)

from scihub import SciHub
import pandas as pd
sh = SciHub()
pickle_file = '2023-7-3_11-24-45_metadata.pkl'
df = pd.read_pickle(pickle_file)
dois = df['DOI'][:20]

for doi, i in zip(dois, range(len(dois))):
    sh.download(str(doi), path=f'/home/er/Documents/Cirad/SOCSciCompiler/data/test_pdf_{i}.pdf')

INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1007/s10661-016-5663-y (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1002/2013JG002494 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1139/X08-151 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1016/S0883-2927(97)00048-6 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1007/s10533-013-9908-3 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1021/acssuschemeng.9b02150 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1002/2016JG003370 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with identifier 10.1016/j.jconhyd.2018.11.011 (resolved url None) due to request exception.
INFO:Sci-Hub:Failed to fetch pdf with id

In [1]:
from scihub import SciHub
sh = SciHub()
print(SciHub._get_available_scihub_urls(sh))

['https://sci-hub.se/']
